In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
with open('intents.json') as f:
    conversation = json.load(f)

print(conversation)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Hello', 'Are you there?', 'Good day', 'Heya'], 'responses': ['Heyy!', 'Hello!', 'Hi buddy, what can I do for you?', 'Hii!']}, {'tag': 'day greeting', 'patterns': ['Good morning', 'Good afternoon', 'Good evening'], 'responses': ['Good Morning!', 'A lovely morning to you too!', 'Good afternoon!', 'Good evening!', 'Good evening, hope you have a wonderful one.']}, {'tag': 'goodbye', 'patterns': ['Bye', 'Goodbye', 'See ya', 'See you later'], 'responses': ['Bye, have a great day!', 'See you, hope I was of some help to you.', 'Bye!']}, {'tag': 'help', 'patterns': ['What can you help me with?', 'What do you assist me with?', 'What can you do for me?'], 'responses': ['I can help with you with company. Please do not consider me as a proofessional opinion. However I can be a great emotional friend to you.']}, {'tag': 'Starting', 'patterns': ['I wanted to share something with you', 'Can I tell you something?', 'Can you help me with this?'

In [ ]:
for i in conversation['intents']:
  print(i['tag'])
  print(i['patterns'])
  print(i['responses'])


greeting
['Hi', 'Hey', 'Hello', 'Are you there?', 'Good day', 'Heya']
['Heyy!', 'Hello!', 'Hi buddy, what can I do for you?', 'Hii!']
day greeting
['Good morning', 'Good afternoon', 'Good evening']
['Good Morning!', 'A lovely morning to you too!', 'Good afternoon!', 'Good evening!', 'Good evening, hope you have a wonderful one.']
goodbye
['Bye', 'Goodbye', 'See ya', 'See you later']
['Bye, have a great day!', 'See you, hope I was of some help to you.', 'Bye!']
help
['What can you help me with?', 'What do you assist me with?', 'What can you do for me?']
['I can help with you with company. Please do not consider me as a proofessional opinion. However I can be a great emotional friend to you.']
Starting
['I wanted to share something with you', 'Can I tell you something?', 'Can you help me with this?']
['Yes please, go ahead.', 'Please do, this is a space space.', 'Yes, I will try my best to understand you.']
Anxious
['I feel slightly anxious these days.', 'I am unable to sleep well these 

In [ ]:
convo_train = []
labels_train = []
labels = []
responses = []
for intent in conversation['intents']:
    for pattern in intent['patterns']:
        convo_train.append(pattern)
        labels_train.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [ ]:
labelencoder = LabelEncoder()
labelencoder.fit(labels_train)
labels_train = labelencoder.transform(labels_train)

In [ ]:
vsize = 1000
dimensions = 16
maxlen = 20
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vsize, oov_token=oov_token)
tokenizer.fit_on_texts(convo_train)
wordin = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(convo_train)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=maxlen)

In [ ]:
model = Sequential()
model.add(Embedding(vsize, dimensions, input_length=maxlen))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 11)                187       
                                                                 
Total params: 16,731
Trainable params: 16,731
Non-trainable params: 0
__________________________________________________

In [ ]:
hist = model.fit(padded_sequences, np.array(labels_train), epochs=1000)

Epoch 1/1000
2/2 [==============================] - 1s 10ms/step - loss: 2.3987 - accuracy: 0.0208
Epoch 2/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.3969 - accuracy: 0.0625
Epoch 3/1000
2/2 [==============================] - 0s 10ms/step - loss: 2.3956 - accuracy: 0.1875
Epoch 4/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.3943 - accuracy: 0.1875
Epoch 5/1000
2/2 [==============================] - 0s 9ms/step - loss: 2.3932 - accuracy: 0.1667
Epoch 6/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.3920 - accuracy: 0.1667
Epoch 7/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.3908 - accuracy: 0.2083
Epoch 8/1000
2/2 [==============================] - 0s 14ms/step - loss: 2.3900 - accuracy: 0.2083
Epoch 9/1000
2/2 [==============================] - 0s 9ms/step - loss: 2.3886 - accuracy: 0.2292
Epoch 10/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.3875 - accuracy: 0.2708
Epoch 11/1000
2/

In [ ]:
model.save("model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(labelencoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)